# DS-HW[2] Data Design and Building

## Problem 2. Data Building

### 1) DB connection check

In [10]:
SERVER_IP = "202.30.21.34"
DB_NAME = "db201921041"
# 데이터 베이스 연결을  위한 ip 와 데이터베이스 이름

In [11]:
import logging
import pymysql

import socket

def _CheckIP():
    return socket.gethostbyname(socket.gethostname()) == SERVER_IP

def _getHostName():    
    if _CheckIP():
        return "localhost"
    else:
        return SERVER_IP
        
def connectDb(user, password):
    host = _getHostName()

    try:
        return pymysql.connect(host = host, user = user, password = password, db = DB_NAME, charset="utf8")
    except:
        logging.error("Check DB info")
        logging.error("user : " + user + ", password : " + password)        
# 실습때 진행한 데이터 베이스연결 코드        

In [12]:
conn = connectDb("std201921041", "201921041")
cur = conn.cursor()
# 데이터 접근을 위한 cursor 지정 및 Db 연결

위 부분은 실습때 진행했던 Db 연결 및 접속에 관한 코드이다.

### 2) DB insertion Process (Tuple Insertion)

In [13]:
import pandas as pd

data= pd.read_csv("dataset/dbACC.csv") #csv 파일 불러 오기
tupleData = [tuple(x) for x in data.values] #튜플화 시킨후 저장

In [53]:
tupleData #데이터 확인

[(0.0, -0.2087, -0.4806, 0.377),
 (1.0, -0.2104, -0.4819, 0.3759),
 (2.0, -0.2103, -0.4822, 0.3798),
 (3.0, -0.2119, -0.4819, 0.3806),
 (4.0, -0.2148, -0.4812, 0.3823),
 (5.0, -0.2156, -0.4812, 0.3823),
 (6.0, -0.2185, -0.4817, 0.3839),
 (7.0, -0.2201, -0.4805, 0.3848),
 (8.0, -0.2209, -0.4796, 0.3862),
 (9.0, -0.2227, -0.4794, 0.3859),
 (10.0, -0.2253, -0.4786, 0.3881),
 (11.0, -0.2253, -0.4792, 0.3879),
 (12.0, -0.2278, -0.4785, 0.3871),
 (13.0, -0.2302, -0.4767, 0.387),
 (14.0, -0.2312, -0.4758, 0.3872),
 (15.0, -0.2322, -0.4742, 0.3883),
 (16.0, -0.2339, -0.4727, 0.387),
 (17.0, -0.2349, -0.4722, 0.3899),
 (18.0, -0.2363, -0.4722, 0.3936),
 (19.0, -0.2373, -0.4724, 0.3955),
 (20.0, -0.2363, -0.4733, 0.3955),
 (21.0, -0.2373, -0.4749, 0.3945),
 (22.0, -0.2374, -0.4745, 0.3987),
 (23.0, -0.2383, -0.4731, 0.4019),
 (24.0, -0.24, -0.4714, 0.4059),
 (25.0, -0.241, -0.4692, 0.405),
 (26.0, -0.2393, -0.4705, 0.4037),
 (27.0, -0.2393, -0.4691, 0.4045),
 (28.0, -0.2397, -0.4692, 0.4055),
 (

위 부분은 Csv 파일을 불러오고, Db에 적용시키기 위하여 튜플화를 시킨 후 저장한 것이다.

In [5]:
#sensor table
query= "insert into sensor (Sensor_id,sensor_type) values('1','accelerator')" #센서 테이블에 입력하는 쿼리, sensor_id 와 sensor_type에 1과 accelerator을 넣는다
cur.execute(query) #쿼리 실행하기
conn.commit() # Db에 적용


센서 테이블 삽입 과정이다. 실습과정을 기반으로 작성했으며, accelerator 하나만 센서가 존재하기 때문에, sensor_id를 1, sensor_type을 accelerator로 지정해서 쿼리를 실행했고, 이를 데이터 베이스에 적용했다.

In [6]:
#acceelerator 
query = "insert into accelerator (nSeq, PORT4_CHN1, PORT6_CHN1, PORT7_CHN1, sensor_id) values ( %s, %s, %s, %s, '1')" #accelerator 테이블에 입력, 센서와 동일하지만 attributes가 다르며, sensor_id 는 sensor table을 참조하며 1이라 디폴트 해놓는다.
cur.executemany(query, tupleData) #쿼리 실행
conn.commit() #적용



다음은 accelerator 테이블 삽입 과정인데, 센서와 큰 차이는 없고 실습과도 유사하지만, sensor_id가 추가 되었다는 차이가 있다. accelerator의 id 는 센서
테이블을 참고하면 1인 것을 알 수 있고, accelerator 안의 sensor_id 는 모두 1로 지정하면 된다. 그리고 나머지 attributes 에 대한 값은 위에서 csv 파일
을 통해 불러온 튜플을 지정해 삽입해주었다. 
sensor_id 가 Sensor 테이블의 Sensor_id를 참조했다 할 수 있다.

In [7]:
#register
registerTuple =[('1','PORT4_CHN1'),('2','PORT6_CHN1'),('3','PORT7_CHN1')] 
registerTuple
#register 테이블에 넣기 위해 포트 4 6 7 각각 1,2,3으로 묶어 튜플화 시킨다.

[('1', 'PORT4_CHN1'), ('2', 'PORT6_CHN1'), ('3', 'PORT7_CHN1')]

In [8]:
query = "insert into register (register_id, register_name, sensor_id) values (%s, %s, '1')" # sensor_id attributes가 있기 때문에, 이 부분은 1로 해놓는다.
cur.executemany(query,registerTuple) #%s 부분에는 위에 만든 registerTuple 데이터가 각각 들어가고 실행된다.
conn.commit()# Db에 적용

레지스터 테이블 삽입 과정이다. 마찬가지로 register_id, register_name을 넣어주기 위해 registerTuple을 지정해주어 넣어 준 것을 확인 할 수있다.
포트 4 6 7에 각각 register_id 는 1 2 3이 지정되고, 구현 사항에서 요구한 register 테이블과 센서 테이블의 연결을 하기 위해 레지스터 테이블
attributes 에 sensor_id 를 추가한다. 레지스터 테이블 같은 경우 accelerator 의 포트 4 6 7에 대한 내용이기 때문에, 논리 상 sensor_id 또한 1이라 할 수 있다. 
레지스터 테이블 내의 sensor_id 가 sensor 테이블의 Sensor_id 를 요구사항에 맞게 참조했다고 할 수 있다.

In [9]:
#timewindow

import numpy as np
import random #난수 발생을 하기 위해서 라이브러리 임포트
for i in range(300): #윈도우 인덱스를 300개 만들기 때문에 다음과 같이 300 반복
    x1= np.random.randint(0,30451,size=2) #2개의 난수를 추출하며, 범위는 data_idx 내로 설정한다. 다만 randint 함수 사용법에 따라 끝 부분에 1을 더해줘 30451로 한다.
    while(x1[0]==x1[1]): 
        x1=np.random.randint(0,30451,size=2) #다음과 같은 while 문은 2개의 난수를 뽑았을 떄 같은 난수가 나올 수 있기 때문에, 그 부분을 방지하는 반복문이다.
    x1.sort()  #작은 값-> 큰 값 순으로 x1을 정렬한다.
    print(x1) 
    
    query= "insert into timewindow (start_dataIdx,end_dataIdx) values (%s,%s)" #쿼리 생성, attributes는 다음과 같이 start, end로 구성
    cur.execute(query, (x1[0],x1[1])) # %s 에 각각 x1[0], x1[1]이 들어가며  x1[0],x[1]은 각각 생성된 난수 작은값 큰 값 차례로 들어가 있다.
    conn.commit() # Db에 적용


[ 1283 21903]
[ 2473 10159]
[ 3258 21902]
[ 1442 10362]
[21002 27513]
[12946 13122]
[11730 29073]
[ 5077 17449]
[16133 29312]
[8256 9493]
[ 3606 14389]
[ 7866 16119]
[11609 20823]
[13473 20613]
[14107 28458]
[1497 5499]
[ 4273 22349]
[ 3525 10014]
[12362 16589]
[4936 9545]
[ 9833 21035]
[ 6116 19991]
[13519 29675]
[1373 7852]
[11109 24158]
[ 3126 22222]
[ 5658 15058]
[ 4747 24502]
[20716 26979]
[ 2207 12490]
[1550 7525]
[20815 30123]
[ 3161 27844]
[16589 25201]
[ 8626 11868]
[13532 25014]
[ 3126 17897]
[ 5761 17498]
[13470 24879]
[ 9618 20592]
[20638 27926]
[ 646 2237]
[13741 21775]
[ 7282 17832]
[ 5051 22974]
[ 8345 29302]
[ 4602 27793]
[ 323 9825]
[20269 21874]
[15363 29920]
[11007 26397]
[10616 20698]
[14293 19036]
[ 3650 20330]
[1619 9136]
[ 2269 10617]
[ 6204 16863]
[ 3699 29008]
[22104 30059]
[27233 27381]
[21725 24610]
[2316 9572]
[ 6327 12199]
[14418 30356]
[ 4342 27480]
[6563 8527]
[ 4403 18407]
[17492 29207]
[ 1591 13189]
[  442 25604]
[ 4647 24762]
[  368 10790]
[16429 26133

다음은 타임윈도우 테이블을 삽입 하는 과정인데, 여기서 부턴 자신이 정한 근거에 따라서 start_idx 와 end_idx를 지정해서 총 300개의 섹션을 만들어야 된다. 
다양한 기준이 있을 수 있겠지만, 본인은 반복문으로 일단 300개를 반복하며 300개의 window_idx를 만들고 각 인덱스마다 랜덤 난수 2개를 data_idx 범위 내의 값 즉 30450 내에서 난수를 2개 뽑는다. 2개가 같은 같으로 나올 수 있기 때문에 while문을 통해 같은 난수 2개가 뽑히지 않게 제약을 걸어주었다.
논리적으로 start_dataIdx < end_dataIdx 여야 하기 때문에 sort를 통해 값을 정렬해 두개의 난 수 중 작은 값이 start ,큰 값이 end로 지정되게 하였다. 이후 각각 쿼리에 넣어주어 타임윈도우 테이블에 넣어준다.

In [10]:
#statistic
query= "SELECT register_name FROM register" #register_name을 찾아와서 
cur.execute(query)
regname= cur.fetchall() #regname에 저장
print(regname) # 프린트 해보기

(('PORT4_CHN1',), ('PORT6_CHN1',), ('PORT7_CHN1',))


In [11]:
query= "SELECT start_dataIdx, end_dataIdx FROM timewindow"
cur.execute(query)
winame=cur.fetchall()
print(winame)

((1283, 21903), (2473, 10159), (3258, 21902), (1442, 10362), (21002, 27513), (12946, 13122), (11730, 29073), (5077, 17449), (16133, 29312), (8256, 9493), (3606, 14389), (7866, 16119), (11609, 20823), (13473, 20613), (14107, 28458), (1497, 5499), (4273, 22349), (3525, 10014), (12362, 16589), (4936, 9545), (9833, 21035), (6116, 19991), (13519, 29675), (1373, 7852), (11109, 24158), (3126, 22222), (5658, 15058), (4747, 24502), (20716, 26979), (2207, 12490), (1550, 7525), (20815, 30123), (3161, 27844), (16589, 25201), (8626, 11868), (13532, 25014), (3126, 17897), (5761, 17498), (13470, 24879), (9618, 20592), (20638, 27926), (646, 2237), (13741, 21775), (7282, 17832), (5051, 22974), (8345, 29302), (4602, 27793), (323, 9825), (20269, 21874), (15363, 29920), (11007, 26397), (10616, 20698), (14293, 19036), (3650, 20330), (1619, 9136), (2269, 10617), (6204, 16863), (3699, 29008), (22104, 30059), (27233, 27381), (21725, 24610), (2316, 9572), (6327, 12199), (14418, 30356), (4342, 27480), (6563, 85

파트 1에서 생각해놨지만, statistic 테이블 같은 경우,  register테이블과  timewindow 테이블을 참조해야될 필요가 있다. 
이를 활용하기 위해 위와 같은 코드를 통해 레지스터 테이블의 registe_name 과 타임 윈도우 테이블의 start end Idx를 불러와서 각각 regname, winame에
저장한다. 출력을 확인해보면, regname같은 경우 포트 4 6 7 이 들어온 것을 확인할 수 있고, winame같은 경우 window_idx 크기인 300개 만큼의 start_Idx end_Idx 쌍이 저장되어 출력되는 것을 확인할 수 있다.

In [12]:
for i in regname: #위에서 저장한 regname 탐색
    if i ==regname[0]:# 포트 4일 경우
        idx=1 #window_idx를 위한 idx 1 선언
        for w in winame: #반복문을 통해 start-end Idx 쌍 탐색
            query_select = "SELECT AVG(PORT4_CHN1), SUM(PORT4_CHN1), COUNT(PORT4_CHN1) FROM accelerator WHERE data_idx BETWEEN %s AND %s"
            #포트 4의 AVG SUM COUNT 값을 선택한다. accelerator의 data_idx 중 밑에 execute 문에 보이지만 w[0] w[1] 만큼의 범위, start-endIdx 내의 범위로 한정한다.
            cur.execute(query_select,(w[0],w[1])) 
            result=cur.fetchall() #result 에 저장한다.
            query_insert= "insert into statistic ( mean, sum, count, reg_id, window_idx) values (%s, %s, %s, %s, %s)"#스태티스틱 테이블에 넣기 위해 쿼리 생성
            cur.execute(query_insert, (result[0][0], result[0][1], result[0][2],'1',idx)) # mean sum count 경우 result에 저장한 값을 가져오면 되고, reg_id는 포트 4이기 때문에 1, idx는
            idx+=1 #idx 같은 경우는 winame 만큼 합산되니 1-300까지 존재하고, 이것이 window_idx가 된다.
    elif i ==regname[1]: #포트 6일 경우 이며 위와 동일
        idx=1
        for w in winame:
            query_select = "SELECT AVG(PORT6_CHN1), SUM(PORT6_CHN1), COUNT(PORT6_CHN1) FROM accelerator WHERE data_idx BETWEEN %s AND %s"
            cur.execute(query_select,(w[0],w[1]))
            result=cur.fetchall()
            query_insert= "insert into statistic ( mean, sum, count, reg_id, window_idx) values (%s, %s, %s, %s, %s)"
            cur.execute(query_insert, (result[0][0], result[0][1], result[0][2],'2',idx))
            idx+=1        
    elif i ==regname[2]:#포트 7일 경우이며, 위와 동일
        idx=1
        for w in winame:
            query_select = "SELECT AVG(PORT7_CHN1), SUM(PORT7_CHN1), COUNT(PORT7_CHN1) FROM accelerator WHERE data_idx BETWEEN %s AND %s"
            cur.execute(query_select,(w[0],w[1]))
            result=cur.fetchall()
            query_insert= "insert into statistic ( mean, sum, count, reg_id, window_idx) values (%s, %s, %s, %s, %s)"
            cur.execute(query_insert, (result[0][0], result[0][1], result[0][2],'3',idx))
            idx+=1
    conn.commit() #Db에 적용한다.

가장 까다로웠던 statistic 테이블이다. 먼저 테이블을 구성할 때 포트 별로 avg sum count attributes 를 구성해 나눠서 구축해볼까 생각했지만, 그렇게 될경우
불필요하게 attributes가 늘어나고, avg sum count 세개로 진행해도, reg_id 참조를 통해 포트 이름을 구분할 수 있기 때문에 위와 같은 방식으로 구축하기로 선택했다. 위에서 저장한 regname을 돌면서 포트 별로 탐색을 했으며, idx를 각 포트별로 반복문에 넣어주며 1-300까지 구성하게 하여 window_idx를 나타내게 하였다. result 부분을 통해서 레지스터 별 avg sum count을 저장했으며, query_insert를 통해 result 값을 넣어줬다. reg_id 는 포트 별로 1,2,3 idx는 순차대로 1-300 1-300 1-300이 반복되어 총 900개의 열이 생기게 된다. 데이터를 db에 넣기 전 외래키로 설정한 데이터를 가져오고, reg_id 의 경우 레지스터 테이블의 register_id 를 참조, window_idx 같은 경우 timewindow 테이블의 window_idx를 참조했다고 할 수 있다. 레지스터 아이디 별로 레지스터 네임이 달라 이를 분류하는 부분과 윈도우 idx를 레지스터 별로 각각 넣어줘 레지스터 별로 윈도우 만큼 1-300까지 나타나게 하는 부분이 생각이 필요하였다. 


### 3) Code to check the contents of each table

- Accelerator
- Sensor
- Register
- Timewindow
- Statistic

In [13]:
#sensor
query = "SELECT * FROM sensor"
cur.execute(query)
senresult= cur.fetchall()
pd.DataFrame(senresult,columns=['Sensor_id','sensor_type'])

,Sensor_id,sensor_type
0,1,accelerator


위는 sensor테이블을 확인하는 부분이며, 가시적으로 보기 좋게 하기 위해서 데이터 프레임화 시켜 확인해보았다.
잘 들어가 있는 것을 확인할 수 있다.

In [14]:
#acceleratior
query = "SELECT * FROM accelerator"
cur.execute(query)
accresult= cur.fetchall()
pd.DataFrame(accresult,columns=['data_idx','nSeq', 'PORT4_CHN1','PORT6_CHN1','PORT7_CHN1','sensor_id'])


,data_idx,nSeq,PORT4_CHN1,PORT6_CHN1,PORT7_CHN1,sensor_id
0,1,0,-0.2087,-0.4806,0.3770,1
1,2,1,-0.2104,-0.4819,0.3759,1
2,3,2,-0.2103,-0.4822,0.3798,1
3,4,3,-0.2119,-0.4819,0.3806,1
4,5,4,-0.2148,-0.4812,0.3823,1
...,...,...,...,...,...,...
30445,30446,30445,-0.0059,-0.6162,0.1777,1
30446,30447,30446,-0.0034,-0.6157,0.1780,1
30447,30448,30447,-0.0037,-0.6145,0.1772,1
30448,30449,30448,0.0000,-0.6157,0.1820,1


위는 accelerator 테이블을 확인하는 부분이며, 30450만큼 추가되어 잇는 것을 알 수 있고, 실습과는 다르게 sensor_id가 1로 추가되어 있는 것을 확인할 수 있다.

In [15]:
#register
query = "SELECT * FROM register"
cur.execute(query)
regresult= cur.fetchall()
pd.DataFrame(regresult,columns=['register_id','register_name','sensor_id'])

,register_id,register_name,sensor_id
0,1,PORT4_CHN1,1
1,2,PORT6_CHN1,1
2,3,PORT7_CHN1,1


위는 레지스터 테이블을 확인한 부분이며, 포트 4 6 7 에 각각 register_id로 1 2 3 이 지정되어있는 것을 확인할 수 있다.

In [16]:
#statistic
query = "SELECT * FROM statistic"
cur.execute(query)
staresult= cur.fetchall()
pd.DataFrame(staresult,columns=['statistic_idx','mean','sum','count','reg_id','window_idx'])

,statistic_idx,mean,sum,count,reg_id,window_idx
0,1,0.020789,428.6940,20621,1,1
1,2,0.059261,455.5410,7687,1,2
2,3,0.017163,320.0050,18645,1,3
3,4,0.049976,445.8380,8921,1,4
4,5,-0.003924,-25.5547,6512,1,5
...,...,...,...,...,...,...
895,896,-0.159209,-1105.5500,6944,3,296
896,897,-0.085167,-536.3850,6298,3,297
897,898,-0.172024,-887.3000,5158,3,298
898,899,-0.121898,-1101.8300,9039,3,299


statistic 테이블을 확인한 부분이다. 위에서 설명했듯이, reg_id 를 보면 처음에는 포트 4를 돌기 때문에 1이며 window_idx 또한 1로 시작한다. 이후 window_idx 가 300이 되면 그 다음 window_idx는 1로 초기화 되며, reg_id는 2로 바뀐다. 즉 포트 6를 돌며 mean sum count값을 도출하는 것이다. 이후 같은 루트대로 돌아 포트 7까지 돌게 된다. 따라서 statistic_idx 는 포트 3개와 각 포트 탐색마다 윈도우 인덱스 만큼인 300개를 돌기 때문에 3*300
즉 900개의 열이 나오게 된다.

In [17]:
#timewindow
query = "SELECT * FROM timewindow"
cur.execute(query)
timresult= cur.fetchall()
pd.DataFrame(timresult,columns=['window_idx','start_dataIdx','end_dataIdx'])

,window_idx,start_dataIdx,end_dataIdx
0,1,1283,21903
1,2,2473,10159
2,3,3258,21902
3,4,1442,10362
4,5,21002,27513
...,...,...,...
295,296,18682,25625
296,297,787,7084
297,298,22628,27785
298,299,15003,24041


timewindow 테이블을 확인한 부분이다. 중점적으로 볼 부분은 start_dataIdx, end_dataIdx가 난수로 2개 생성되어 총 window_idx가 300개 원하는 조건대로 알맞게 잘 들어간 것을 볼 수 있다. 또한 sort를 사용했기 때문에 start 에 지정된 값이 end 에 지정된 값보다 항상 작은 것을 확인할 수 있다.

In [107]:
conn.close()

### 4) (Free) Search table

After defining five arbitrary conditions, search for tuples that satisfy the condition in the statistic table.

A) Condition 1

 본인만의 방식을 정해 자유롭게 5가지의 조건을 statistic table에 적용한다. 조건을 만족하는 데이터를 찾아 몇개 존재하는지 확인해 보았다.

A 방법)레지스터 전체 평균< 윈도우 인덱스별 평균인 윈도우의 개수
즉 각 레지스터의 전체 평균보다 큰 윈도우의 개수 탐색

In [101]:
query= " SELECT AVG(PORT4_CHN1), AVG(PORT6_CHN1), AVG(PORT7_CHN1) FROM accelerator WHERE data_idx " #accelerator 테이블 내에서 포트 4, 6, 7 각각의 평균값을 구한다.
#각 레지스터 별로 인덱스 범위는 전체로 구성한다.
cur.execute(query) #쿼리 실행
result= cur.fetchall() #resultdp 도출된 값 저장
print(result)
print(result[0])
print(result[0][1])# 확인

OperationalError: (2006, "MySQL server has gone away (BrokenPipeError(32, 'Broken pipe'))")

위 코드 보고서 쓰면서 실행을 잘못 눌러서 오류 떠있는걸로 나오는데, 실행 잘되는 코드입니다..!!

In [53]:
for i in result[0]: # 레지스터 별로 돈다
    if i == result[0][0]:#포트 4의 평균 값을 돌때 
        query = "SELECT COUNT(mean) FROM statistic WHERE reg_id='1' AND mean>%s" #statistic 테이블 내의 mean column 내의 값을 카운트한다. 대신 제약 조건은 reg_id=1이며, 포트 4의 전체 편균보다 큰 데이터를 가진 mean column의 인덱스 여야지만
        #카운트한다.
        print("accelerator port4의 평균 값인 -0.00078보다 큰 평균을 가지고 있는 윈도우의 개수") 
        cur.execute(query,result[0][0]) #%s 에 넣을 값은 result[0][0]즉 포트 4의 전체 평균 값이다.
        
    elif i == result[0][1]:#포트 6의 평균 값을 돌때, 방식은 위와 동일
        query = "SELECT COUNT(mean) FROM statistic WHERE reg_id='2' AND mean>%s"
        print("accelerator port6의 평균 값인 0.421보다 큰 평균을 가지고 있는 윈도우의 개수")
        cur.execute(query,result[0][1])  
    elif i == result[0][2]:#포트 7의 평균 값을 돌때, 방식은 위와 동일
        query = "SELECT COUNT(mean) FROM statistic WHERE reg_id='3' AND mean>%s"
        print("accelerator port7의 평균 값인 -0.070보다 큰 평균을 가지고 있는 윈도우의 개수")      
        cur.execute(query,result[0][2]) 
       
    result1=cur.fetchall() #나온 결과를 result1에 저장
    print(result1[0])#프린트 해보기
        

accelerator port4의 평균 값인 -0.00078보다 큰 평균을 가지고 있는 윈도우의 개수
(190,)
accelerator port6의 평균 값인 0.421보다 큰 평균을 가지고 있는 윈도우의 개수
(163,)
accelerator port7의 평균 값인 -0.070보다 큰 평균을 가지고 있는 윈도우의 개수
(66,)


A 방법에서는 먼저 accelerator의 토프 4 6 7에 대한 각각의 레지스터 당 평균을 도출해냈다. 이를 이용해서 각 레지스터 별로 반복문을 돌때 statistic 내의 mean column 안의 데이터와 비교해서 mean column안 데이터가 더 크다면, 카운트해서 저장하게 했다. 또한 reg_id 를 통해 포트별로 나눠서 판단할 수 있도록 제약 조건을 걸었다. 결과를 보았을 때 포트 4에서는 190개, 포트 6에서는 163개 포트 7에서는 66개가 존재했다.
각 레지스터 마다 300개의 인덱스가 존재하기 때문에 비율로 따지면 각각 63%,  54.3%, 22%다. 
이를 통해 확인할 수 있는 점은 statistic 테이블에서 윈도우를 통해 파싱된 포트 4 6 은 평균보다 큰 값들이 많이 존재하고 있다는 점을 알 수 있고 포트 7 같은 경우 -0.07 보다 작은 값이 많다 생각할 수 있으며 포트 7에는 음수인 데이터 값이 많이 존재한다 추측할 수 있다. 물론 window의 start-end idx 자체를 random하게 뽑아내었기 때문에, 정확한 분석이라 할 수 없다. 그렇기 때문에 방금 가정한 포트 7에는 음수인 부분이 많이 존재하는지 알아보기 위해 밑에서 다른 가정을 통해 분석을 진행해보겠다.

B) Condition 2

각 레지스터 별로 윈도우 구간 합이 0보다 큰 윈도우 개수

In [56]:
reg_id =[1,2,3]
reg_id[0]

1

In [57]:
for i in reg_id:
    if i == reg_id[0]: #포트 4를 돌때
        query = "SELECT COUNT(sum) FROM statistic WHERE reg_id='1' AND sum>0" #이번에는 sum column의 데이터 값을 count 하는데, 제약조건은
        # reg_id 와 sum column 의 데이터 값이 0보다 클 경우에만 카운트 하는 것이다.
        print("statistic 테이블 port4 윈도우 구간 합이 0보다 큰 윈도우의 개수")
        cur.execute(query) 
        
    elif i == reg_id[1]:#포트 6을 돌때 , 밑은 위와 동일
        query = "SELECT COUNT(sum) FROM statistic WHERE reg_id='2' AND sum>0"
        print("statistic 테이블 port6 윈도우 구간 합이 0보다 큰 윈도우의 개수")
        cur.execute(query)  
    elif i == reg_id[2]: #포트 7을 돌때, 밑은 위와 동일
        query = "SELECT COUNT(sum) FROM statistic WHERE reg_id='3' AND sum>0"
        print("statistic 테이블 port7 윈도우 구간 합이 0보다 큰 윈도우의 개수")      
        cur.execute(query) 
       
    result1=cur.fetchall() #카운트 해서 나온 결과를 result1에 저장하고 프린트
    print(result1[0])

statistic 테이블 port4 윈도우 구간 합이 0보다 큰 윈도우의 개수
(187,)
statistic 테이블 port6 윈도우 구간 합이 0보다 큰 윈도우의 개수
(278,)
statistic 테이블 port7 윈도우 구간 합이 0보다 큰 윈도우의 개수
(14,)


이번에는 sum column 내의 데이터 값을 활용하여 탐색을 진행해 보았다. 일단 먼저 sum column 내의 데이터 같은 경우 레지스터 별로 나눠져있으며 각각 300개의 윈도우 섹션이 존재하여 각 인덱스 마다 랜덤으로 파싱한 구간 내의 데이터들을 합친 값이 존재한다.
코드 분석은 위의 주석을 통해 전반적으로 진행했으며, 결과를 분석해보면, 포트 4에서는 제약 조건을 만족하는 윈도우의 갯수가 187개 포트 6에서는 278개 이며, 포트 7에서는 14개 밖에 존재하지 않는다. 
비율로 확인하면 포트 4는 62.3% 포트 6는 92.6% 포트 7은 4.6% 정도이다. 포트 7이 압도적으로 낮은 것을 알 수 있다.
다음과 같은 제약 조건을 통해 도출할 수 있는 점은, A 조건과 연계하여 생각해봤을 때 포트 7에는 음수인 데이터가 상대적으로 훨씬 많이 존재하기 때문에 다음과 같은 제약 조건 즉 sum 을 하게될 경우 0보다 훨씬 작아질 수 밖에 없다. 랜덤으로 구간을 파싱해 윈도우 인덱스 별로 구간을 설정했지만 다음과 같은 근거를 들어 본인의 가정에 힘을 실을 수 있다. B 조건을 진행하면서 랜덤으로 구간을 파싱했기 때문에, 각 윈도우 인덱스 마다 start-end 사이 구간도 천차 만별로 다를 수 있을 거 같다는 생각이 들었고, 이는 분석과 가정에 영향을 줄 수 있다는 생각이 들어 구간을 제약 조건으로 걸어 알아보는 분석을 진행하도록 하겠다.

C) Condition 3

각 레지스터 별로, count의 평균 값을 내고 평균 count의 크기보다 큰 count를 가지는 윈도우의 개수

In [65]:
for i in range(3):
    query= "SELECT AVG(count) FROM statistic WHERE reg_id=%s"
    cur.execute(query,i+1)
    result= cur.fetchall()
    print(result[0])

(Decimal('10779.3567'),)
(Decimal('10779.3567'),)
(Decimal('10779.3567'),)


각 레지스터 별로 count의 평균을 내보았는데, 레지스터는 다르지만 파싱한 윈도우 인덱스 별 구간은 동일하기 때문에, 결과가 레지스터별로 다 똑같은 점을 확인할 수 있다.

In [67]:

    query = "SELECT COUNT(count) FROM statistic WHERE reg_id='1' AND count>%s" # 레지스터가 달라도 같은 값이 도출될거기 때문에 포트 4를 기준으로 하나만 파악해보았다.
    # 이번에는 count 열의 데이터들을 카운트했으며, 제약조건은 위에서 저장한 result[0] 보다 클 경우에만 카운트한다.
    print("평균 count의 크기보다 큰 count를 가지는 윈도우의 개수")
    cur.execute(query,result[0]) 
    result1=cur.fetchall()
    print(result1[0])

평균 count의 크기보다 큰 count를 가지는 윈도우의 개수
(135,)


코드 분석은 주석을 통해 대체하겠다. 이번에는 count 열을 이용한 분석을 진행해보았다. count 의 평균 값을 먼저 계산 해보았는데, 10779 정도가 나왔다. 이는 
파싱된 윈도우 인덱스 에서 start-end 사이 구간 값의 평균이 10779 라는 것이다. 랜덤으로 start-end idx를 구성했기 때문에 인덱스 별로 천차 만별 구간 값이 존재 할 것이다. 평균 구간 값보다 큰 구간 값을 가지는 윈도우 인덱스의 개수를 구했을 때 135개 
비율로 따지면 45% 정도이다. 평균보다 작은 구간을 가지는 인덱스의 비율이 55%라는 점에서 고르게 분포했다고 생각할 수 있지만, 앞서 말햇듯이 구간 크기가 천차 만별이기 때문에 스태티스틱 테이블의 mean sum 값에 영향을 줄 수 있기 때문에 statistic 테이블의 mean sum 을 가지고 가정 혹은 분석을 진행할 때 섣부르게 판단 할 수 없을 것이다. 이왁 같은 상황을 막기 위해서 뒤에 다시 설명하겠지만 start-end idx를 선택하는 부분에 있어서 구간을 특정 값으로 진행해서 파싱을 진행했으면 좋을 것이다. 
마지막으로 C 분석을 통해 대략적인 파싱 구간의 크기를 예측할 수 있었다.

D) Condition 4

각 레지스터별 sum을 해 값을 도출하고, 그 값보다 큰 sum을 가진 윈도우 개수

In [68]:
query= " SELECT SUM(PORT4_CHN1), SUM(PORT6_CHN1), SUM(PORT7_CHN1) FROM accelerator WHERE data_idx "
cur.execute(query)
result= cur.fetchall()
print(result)
print(result[0])
print(result[0][1])

((-23.759199938402162, 12834.97818954051, -2138.1269905019944),)
(-23.759199938402162, 12834.97818954051, -2138.1269905019944)
12834.97818954051


위의 코드는 accelerator 테이블의 data_idx에서 레지스터 별로 각각 값들을 더해주고 이를 프린트한 결과이다. 포트 4는 -23.759, 포트 6는 12834, 포트 7은 -2138.12699 인데, 이를 보고 포트 7에 압도적으로 음수가 많다는 가정을 뒷받침할 수 있다.

In [69]:
for i in result[0]:
    if i == result[0][0]: #포트 4일때
        query = "SELECT COUNT(sum) FROM statistic WHERE reg_id='1' AND sum>%s" #제약조건 reg_id =1 이고 sum 열 안의 데이터 값이 포트 4의 합보다 클 경우만 카운트
        print("port 4의 sum 값인 -23.759보다 큰 sum을 가지고 있는 윈도우의 개수")
        cur.execute(query,result[0][0]) 
        
    elif i == result[0][1]: #포트 6일때 
        query = "SELECT COUNT(sum) FROM statistic WHERE reg_id='2' AND sum>%s" #제약조건 reg_id =2 이고 sum 열 안의 데이터 값이 포트 6의 합보다 클 경우만 카운트
        print("port 6의 sum 값인 12834.978보다 큰 sum을 가지고 있는 윈도우의 개수")
        cur.execute(query,result[0][1])  
    elif i == result[0][2]: #포트 7일때 
        query = "SELECT COUNT(sum) FROM statistic WHERE reg_id='3' AND sum>%s"#제약조건 reg_id =3 이고 sum 열 안의 데이터 값이 포트 7의 합보다 클 경우만 카운트
        print("port 7의 sum 값인 -2138.1269보다 큰 sum을 가지고 있는 윈도우의 개수")      
        cur.execute(query,result[0][2]) 
       
    result1=cur.fetchall() #저장
    print(result1[0])#확인
        

port 4의 sum 값인 -23.759보다 큰 sum을 가지고 있는 윈도우의 개수
(202,)
port 6의 sum 값인 12834.978보다 큰 sum을 가지고 있는 윈도우의 개수
(25,)
port 7의 sum 값인 -2138.1269보다 큰 sum을 가지고 있는 윈도우의 개수
(270,)


코드 구현 방식은 그 동안 위에서 해왔던 방법과 같아서 자세히 다루지 않겠다. 제약 조건이 sum>result[0][0], sum>result[0][1], sum>result[0][2] 를 만족하는 경우에만 카운트 된다는 부분이 특징이다. statistic 테이블 안에 있는 sum column의 데이터 값 같은 경우, 랜덤으로 파싱한 윈도우 구간들 범위안의 값들을 더한 값이 존재하는 것이다. 결과 분석을 해보자면 먼저 조건을 만족하는 윈도우의 개수는 각각 202개 25개 270개 이며 비율로 따지면
포트 4는 67.3% 포트 6는 8.3%, 포트 7은 90%이다. 이번 결과에서는 의외로 포트 6의 개수가 현저히 적었는데, port 6의 전체 sum 값이 12834.978 로 크기 때문이다. 이와 반대로 포트 7의 경우 sum의 값이 -2138 정도로 매우 작기 때문에 90프로 가 조건에 만족하는 것을 확인할 수 있었다. 이를 통해 포트 6가 다른 포트들에 비해 양수가 많다고 짐작할 수 있다. 또한 레지스터 별로 전체 데이터 들을 더한 값과 랜덤으로 파싱한 구간에 대해 더한 값을 비교 한 것이다 보니, 데이터들이 양수로 이뤄져 있었다면, sum column 내의 데이터 구간은 전체 구간보단 작을 수 밖에 없으니 결과적으로 포트 모두 0이 나올 수밖에 없을 것이다. 이러한 결과가 나온 이유는 데이터 값들 사이에 음수가 존재하기 때문에 가능했기 때문이다. 
결론적으로 이 분석을 통해 대략적으로 데이터들 사이 양수와 음수 의 차지도를 짐작 할 수 있다. 
또한 조금 더 구체적으로 분석을 진행해 단순히 개수만 세는 것이 아니라 어떤 구간에서 어떤 값이 나왔는지 볼 수 있다면, 그 구간의 데이터 값들의 분포를 조금 더 구체적으로 파악할 수 있을 것이다.

E) Condition 5

statistic 테이블에서 각 레지스터마다 mean 부분의 평균을 구해보고. 레지스터 별로 나온 평균보다 높은 mean을 가지고 있는 윈도우의 갯수

In [97]:

    query= " SELECT AVG(mean) FROM statistic WHERE reg_id='1' "
    cur.execute(query)
    result= cur.fetchall()
    print(result[0])
    query= " SELECT AVG(mean) FROM statistic WHERE reg_id='2' "
    cur.execute(query)
    result1= cur.fetchall()
    print(result1[0])
    query= " SELECT AVG(mean) FROM statistic WHERE reg_id='3' "
    cur.execute(query)
    result2= cur.fetchall()
    print(result2[0])
    #이번에는 statistic 테이블 내에서 포트별로 평균을 구해보았다.

    result3=[]
    result3.extend(result[0])
    result3.extend(result1[0])
    result3.extend(result2[0])
    print(result3)
    #이를 result3 라는 비어있는 list에 append하여 리스트를 만들어놓았다.

    




(0.011886880883442549,)
(0.5410244418230529,)
(-0.08881151700896832,)
[0.011886880883442549, 0.5410244418230529, -0.08881151700896832]


In [100]:
for i in result3:
    if i == result3[0]:
        query = "SELECT COUNT(mean) FROM statistic WHERE reg_id='1' AND mean>%s"
        print("port 4의 sum 값인 -23.759보다 큰 sum을 가지고 있는 윈도우의 개수")
        cur.execute(query,result3[0]) 
        
    elif i == result3[1]:
        query = "SELECT COUNT(mean) FROM statistic WHERE reg_id='2' AND mean>%s"
        print("port 6의 sum 값인 12834.978보다 큰 sum을 가지고 있는 윈도우의 개수")
        cur.execute(query,result3[1])  
    elif i == result3[2]:
        query = "SELECT COUNT(mean) FROM statistic WHERE reg_id='3' AND mean>%s"
        print("port 7의 sum 값인 -2138.1269보다 큰 sum을 가지고 있는 윈도우의 개수")      
        cur.execute(query,result3[2]) 
       
    resultf=cur.fetchall()
    print(resultf[0])
        

port 4의 sum 값인 -23.759보다 큰 sum을 가지고 있는 윈도우의 개수
(148,)
port 6의 sum 값인 12834.978보다 큰 sum을 가지고 있는 윈도우의 개수
(139,)
port 7의 sum 값인 -2138.1269보다 큰 sum을 가지고 있는 윈도우의 개수
(132,)


마지막 분석이다. 코드는 위의 4가지 방식과 같은 알고리즘으로 진행했기 때문에, 코드 분석은 진행하지 않겠다.
위에서 진행한 분석과 다른 점이 잇다면 기존에는 accelerator 에 있는 레지스터의 전체 값들의 평균을 내서 그 값보다 큰 값보다 큰 윈도우 개수를 파악했다면, 
이번에는 statistic 테이블 내에서 레지스터 별로 mean 인덱스 내의 값들을 평균 내서 이를 기준으로 잡았다.
결과를 보았을때 각각 148 139 132로 비율 상으로 보면 49.3% 46.3% 44%로 그 동안 분석했을때 결과들의 비율 중 50%에 가장 근접한 비율이  나왔다. 
A 조건과 비교했을 때 확실히 E조건은 mean column 내의 데이터들의 값을 또 평균 낸것을 기준 잡고 이 보다 큰 것들을 카운트 한 것이다 보니 A 조건 보다는 정규 분포의 형태를 띄게 된다 생각할 수 있다.


최종분석

지금 까지 여러 조건을 통해서 statistic table을 분석 해보았다. 추정 할 수 있느 부분은 포트 6가 가장 양수가 많이 존재하며, 포트7이 이와 반대로 음수가 가장 많이 존재한다는 부분이다.
정확하다고 단정지어 말할 수 없는 부분은 타임 윈도우 테이블에서 start-end idx 모두 내가 랜덤 함수를 통해 2개 다 난수로 뽑았고, 이에 따라서 각 윈도우 인덱스 별로 구간의 크기, 시작과 끝의 부분이 천차 만별이다. 따라서 통일 되어 있지 않기 때문에 대략적으로 유추만 가능하다는 아쉬움이 존재하였다. 스태티스틱 테이블을 통해서 결론적으로 전체적인 경향을 보기에는 어려움이 있고, 분석을 통해 구간에 대한 특징을 살펴보는데 의의를 둘 수 있을 것 같다.(음수 양수의 분포 등등..) 이러한 분포를 조금 더 정확하게 분석 하기 위해서는 방금 본인이 진행했던 것처럼 랜덤 함수를 통해  구간을 잡기 보다는 통일된 구간의 크기를 잡으면 더 좋을 것 같다. 에를 들어 start-end idx 모두 랜덤으로 잡은 것이 아니라, start idx만 랜덤으로 잡고 픽스되어 있는 크기 (예를 들어 300) 뒤에 있는 인덱스를 end 로 잡아 구간을 통일 했으면 분석 시에 조금 더 직관적으로 구간마다의 분포를 확인하는데 수월하지 않았을까 한다. 
하지만 위 진행을 통해 결론 나온 정보 또한 실제 분포와 비교했을 때 비슷한 결과를 도출한다는 것을 확인 할 수 있었다.
